In [1]:
from time import time
begin = time()

In [2]:
# !pip install flash | grep -v 'already satisfied'
!pip install '../input/flashwheels/flash-1.0.3-py3-none-any.whl' | grep -v 'already satisfied'

# !pip install lightning-flash | grep -v 'already satisfied'
!pip install '../input/flashwheels/jsonargparse-4.9.0-py3-none-any.whl' | grep -v 'already satisfied'
!pip install '../input/flashwheels/setuptools-59.5.0-py3-none-any.whl' | grep -v 'already satisfied'
!pip install '../input/flashwheels/docstring_parser-0.15-py3-none-any.whl' | grep -v 'already satisfied'
!pip install '../input/flashwheels/lightning_flash-0.8.0-py3-none-any.whl' | grep -v 'already satisfied'

# !pip install 'lightning-flash[image]' | grep -v 'already satisfied'
!pip install '../input/flashwheels/pystiche-1.0.1-py3-none-any.whl' | grep -v 'already satisfied'
!pip install '../input/flashwheels/pretrainedmodels-0.7.4-py3-none-any.whl' | grep -v 'already satisfied'
!pip install '../input/flashwheels/efficientnet_pytorch-0.7.1-py3-none-any.whl' | grep -v 'already satisfied'
!pip install '../input/flashwheels/timm-0.4.12-py3-none-any.whl' | grep -v 'already satisfied'
!pip install '../input/flashwheels/segmentation_models_pytorch-0.3.0-py3-none-any.whl' | grep -v 'already satisfied'
!pip install '../input/flashwheels/lightning_bolts-0.5.0-py3-none-any.whl' | grep -v 'already satisfied'

# !pip install cucim | grep -v 'already satisfied'
!pip install '../input/flashwheels/click-8.1.3-py3-none-any.whl' | grep -v 'already satisfied'
!pip install '../input/flashwheels/cucim-22.8.1-py3-none-manylinux2014_x86_64.whl' | grep -v 'already satisfied'

Processing /kaggle/input/flashwheels/flash-1.0.3-py3-none-any.whl
Processing /kaggle/input/flashwheels/jsonargparse-4.9.0-py3-none-any.whl
Processing /kaggle/input/flashwheels/setuptools-59.5.0-py3-none-any.whl
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
thinc 8.0.17 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
spacy 3.3.1 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
google-api

In [3]:
from cv2 import resize as cvresize
from cv2 import INTER_NEAREST
from flash import Trainer
from flash.image import ImageClassificationData, ImageClassifier
from joblib import Parallel, delayed
from openslide import OpenSlide
from os import listdir, stat
from PIL import Image
from pytorch_lightning import seed_everything
from skimage.io import imread
from math import ceil
from pandas import DataFrame
from timeit import time as ittime
from torch import tensor, concat, split, device, cuda, no_grad
from tqdm import tqdm
from warnings import filterwarnings

In [4]:
from cupy import array, argsort, concatenate, dstack, ones, repeat, stack, uint8, where, zeros, zeros_like 
from numpy import uint8 as npuint8
from numpy import array as nparray
from numpy import where as npwhere
from numpy import argsort as npargsort
from numpy import concatenate as npconcatenate
from numpy import ones as npones
from numpy import zeros as npzeros
from cucim.skimage.color import rgb2hed, hed2rgb, rgb2gray
from cucim.skimage.exposure import match_histograms
from cucim.skimage.filters import gaussian, threshold_otsu
from cucim.skimage.measure import label, regionprops
from cucim.skimage.morphology import dilation, disk, opening, square
from cucim.skimage.segmentation import clear_border
from cucim.skimage.transform import resize

In [5]:
cuda_device = device("cuda:0" if cuda.is_available else "cpu")

In [6]:
filterwarnings("ignore")

In [7]:
Image.MAX_IMAGE_PIXELS = None
RAM = 11e9
RESOLUTION = 10
SIZE_CAPS = [10e8, 12e8, 15e8, 20e8]
SLIDE_PERC_TAKENS = [1.0, 0.9, 0.8, 0.6] # percentages taken refering to sizes
SKIPPED_SIZE = 2e8 # keep this for the end (usually doesn't contain enought signal)
DATA_DIR = '/kaggle/input/mayo-clinic-strip-ai/test/'

In [8]:
FPVAL = 16
SIGMA = 2
MIN_AREA = 10000
MAX_SLICE_AREA = 25000000 # 5000*5000

In [9]:
TILE_SIZE = 224
TILE_AREA = TILE_SIZE/RESOLUTION*TILE_SIZE/RESOLUTION # for segmentation from preview
PIXEL_CAP = 240*3 # to be considered as white
NOT_BLANK_PERC = 0.95
NOT_BLANK_AREA = TILE_SIZE*TILE_SIZE*NOT_BLANK_PERC

In [10]:
seed_everything(12)
REF_IMAGE = array(imread('../input/harmonizer-ref/har_ref.png'))
PATH_TO_MODEL_WEIGHTS = "../input/mayomodelsv2/decentmodelv3.ckpt"
transform_kwargs = {"image_size": (224, 224), "mean": (0.5, 0.5, 0.5), "std": (0.5, 0.5, 0.5)}
BATCH_SIZE = 64
UNIQUE_LABELS = {"CE":0, "LAA":1}

In [11]:
def get_preview(slide, loaded_perc=1.0):
    width, height = slide.dimensions
    width, height = int(width*loaded_perc), int(height*loaded_perc)
    new_width = int(width/RESOLUTION)
    new_height = int(height/RESOLUTION)
    amount = ceil(2*(12*width*height + new_width*new_height*22)/ RAM)

    w = int(width/amount) # tile width
    h = int(height/amount) # tile height
    nw = int(new_width/amount) # tile new width
    nh = int(new_height/amount) # tile new height

    preview = zeros((new_height, new_width, 3), dtype=uint8) + 255
    for i in range(amount):
        for j in range(amount):
            preview[j*nh:(j+1)*nh, i*nw:(i+1)*nw] = array(cvresize(
                nparray(slide.read_region((i*w,j*h), 0, (w, h)), dtype=npuint8)[:,:,:3], 
                dsize=(nw, nh), interpolation=INTER_NEAREST), dtype=uint8)
    return preview

In [12]:
def get_cleaned_binary(preview):
    bw = rgb2gray(preview)
    bw = threshold_otsu(bw) > gaussian(bw, SIGMA)
    bw = opening(bw, square(FPVAL))
    bw += clear_border(~bw)
    bw = dilation(bw, disk(int(FPVAL*0.25)))
    return bw

In [13]:
my_trainer = Trainer(enable_progress_bar=False, accelerator='gpu', devices=1, fast_dev_run=True)
model = ImageClassifier.load_from_checkpoint(PATH_TO_MODEL_WEIGHTS)

def inference(imgs, batch_size):
    datamodule = ImageClassificationData.from_tensors(predict_data=imgs,
                     transform_kwargs=transform_kwargs,batch_size=batch_size) #dict(image_size=(224, 224)),batch_size=batch_size)
    vec = [UNIQUE_LABELS[label] for label in my_trainer.predict(model, datamodule=datamodule, output='labels')[0]]
    return tensor(nparray(vec)).to(cuda_device)

Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth


In [14]:
def get_ihc_hed(img):
        ihc_hed = (rgb2hed(img))
        null = zeros_like(ihc_hed[:, :, 0])
        ihc_h = hed2rgb(stack((ihc_hed[:, :, 0], null, null), axis=-1))
        ihc_e = hed2rgb(stack((null, ihc_hed[:, :, 1], null), axis=-1))
        ihc_d = hed2rgb(stack((null, null, ihc_hed[:, :, 2]), axis=-1))
        return ihc_h, ihc_e, ihc_d

In [15]:
IHC_H_REF, IHC_E_RED, IHC_D_REF = get_ihc_hed(REF_IMAGE)
del REF_IMAGE

In [16]:
def harmonize(img):
    ihc_h, ihc_e, ihc_d = get_ihc_hed(img)
    del img
    return (dstack((match_histograms(ihc_h, IHC_H_REF)[:, :, 0],
                    match_histograms(ihc_d, IHC_E_RED)[:, :, 1],
                    match_histograms(ihc_e, IHC_D_REF)[:, :, 2])
                    )*255).astype(uint8)

In [17]:
def predict_slide(slide, bw):
    with no_grad():
        predicted_labels = tensor(nparray([])).to(cuda_device)
        regions = [region for region in regionprops(label(bw)) if region.area > MIN_AREA]
        if len(regions) == 0: regions = [region for region in regionprops(label(bw)) if region.area > TILE_AREA]
        if len(regions) == 0: return predicted_labels

        scaled_bboxes = []
        bmasks = []
        for region in regions:
            old_x, old_y = region.bbox[0], region.bbox[1]
            x, y = old_x*RESOLUTION, old_y*RESOLUTION
            old_w, old_h = region.bbox[2] - old_x, region.bbox[3] - old_y
            w, h = region.bbox[2]*RESOLUTION - x, region.bbox[3]*RESOLUTION - y
            extra_x, extra_y = (w % TILE_SIZE), (h % TILE_SIZE)
            x += ceil(extra_x/2)
            w -= extra_x
            y += ceil(extra_y/2)
            h -= extra_y

            if w*h > MAX_SLICE_AREA:
                inf_half_w, sup_half_w = int(w/(TILE_SIZE*2))*TILE_SIZE, ceil(w/(TILE_SIZE*2))*TILE_SIZE
                inf_half_h, sup_half_h = int(h/(TILE_SIZE*2))*TILE_SIZE, ceil(h/(TILE_SIZE*2))*TILE_SIZE
                inf_b_half_w, inf_b_half_h = int(inf_half_w/RESOLUTION), int(inf_half_h/RESOLUTION)
                scaled_bboxes.append((x, y, inf_half_w, inf_half_h))
                bmasks.append(region.image[:inf_b_half_w, :inf_b_half_h])

                scaled_bboxes.append((x+inf_half_w, y, sup_half_w, inf_half_h))
                bmasks.append(region.image[inf_b_half_w:, :inf_b_half_h])

                scaled_bboxes.append((x, y+inf_half_w, inf_half_w, sup_half_h))
                bmasks.append(region.image[:inf_b_half_w, inf_b_half_h:])

                scaled_bboxes.append((x+inf_half_w, y+inf_half_w, sup_half_w, sup_half_h))
                bmasks.append(region.image[inf_b_half_w:, inf_b_half_h:])

            else:
                scaled_bboxes.append((x, y, w, h))
                bmasks.append(region.image)
        del regions

        for (x,y,w,h), bmask in zip(scaled_bboxes, bmasks):

            mask = resize(bmask, (w,h))
            slice = array(nparray(slide.read_region((y,x), 0, (h, w)), dtype=npuint8)[:,:,:3], dtype=uint8)
            for chanel in range(3):
                slice[:,:,chanel] *= mask
                slice[:,:,chanel] += (~mask*255).astype(uint8)
            del mask

            n, p = int(slice.shape[0]/TILE_SIZE), int(slice.shape[1]/TILE_SIZE)
            tiles = slice.reshape((n,TILE_SIZE,p,TILE_SIZE,3))
            del slice

            kept_tiles = []
            for i in range(n):
                for j in range(p):
                    if where((tiles[i,:,j,:].sum(axis=-1) < PIXEL_CAP))[0].shape[0] >= NOT_BLANK_AREA:
                        kept_tiles.append(harmonize(tiles[i,:,j,:]).get())
            kept_tiles = tensor(nparray(kept_tiles))
            if kept_tiles.shape[0] == 0: continue

            for infered_tiles in split(kept_tiles.to(cuda_device).permute(0,3,1,2), BATCH_SIZE):
                predicted_labels = concat([predicted_labels, inference(infered_tiles, infered_tiles.shape[0])])
            del kept_tiles

    return predicted_labels

In [18]:
filenames = nparray([filename for filename in listdir(DATA_DIR) if filename.endswith(".tif")])
sizes = nparray([stat(f'{DATA_DIR}{filename}').st_size for filename in filenames])

In [19]:
indices = npargsort(sizes)

first = npwhere(sizes[indices] > SKIPPED_SIZE)[0]
last = npwhere(sizes[indices] <= SKIPPED_SIZE)[0]
indices = npconcatenate([indices[first], indices[last]])

filenames = filenames[indices]
sizes = sizes[indices]

In [20]:
patient_ids = []
kept_indices = []
for i, filename in enumerate(filenames):
    patient_id = filename[:filename.find('_')]
    if patient_id not in patient_ids:
        patient_ids.append(patient_id)
        kept_indices.append(i)
        
kept_indices = nparray(kept_indices)
filenames[kept_indices] = filenames[kept_indices]
sizes[kept_indices] = sizes[kept_indices]

In [21]:
load_percs = npones(sizes.shape)
for i, size in enumerate(sizes):
    perc_taken = SLIDE_PERC_TAKENS[-1]
    for index_taken, cap in enumerate(SIZE_CAPS):
        if size < cap:
            perc_taken = SLIDE_PERC_TAKENS[index_taken]
            break
    load_percs[i] = perc_taken
del SIZE_CAPS, SLIDE_PERC_TAKENS

In [22]:
percs = npzeros(len(patient_ids)) + 0.5
labelization = DataFrame({'patient_id': patient_ids, 'CE': percs, 'LAA': percs})
del kept_indices, patient_ids, percs

In [23]:
labelization = labelization.sort_values('patient_id')

In [24]:
for row, (filename, load_perc, slide_size) in enumerate(zip(filenames, load_percs, sizes)):
    first_start = ittime.time_ns()
    print("--------------------------------------------------------------------")
    print(f"{round(slide_size/1e9, 2)}Go {filename}")
    if slide_size > 35e8:
        continue
    if time() - begin > 30600: # 8h30
        break
    slide = OpenSlide(f'{DATA_DIR}{filename}')
    print(f"Openslide took {round((ittime.time_ns() - first_start)/10e8, 5)} seconds")
    start = ittime.time_ns()
    preview = get_preview(slide, loaded_perc=load_perc)
    print(f"Preview took {round((ittime.time_ns() - start)/10e8, 5)} seconds")
    start = ittime.time_ns()
    bw = get_cleaned_binary(preview)
    print(f"Clean took {round((ittime.time_ns() - start)/10e8, 5)} seconds")
    start = ittime.time_ns()
    del preview
    vec = predict_slide(slide, bw)
    del bw
    print(vec.shape[0])
    if vec.shape[0] > 0:
        perc = 0.99999 if vec.mean() > 0.5 else 0.00001
        labelization.loc[row, 'LAA'] = perc
        labelization.loc[row, 'CE'] = 1 - perc
        labelization.to_csv('/kaggle/working/submission.csv', index=False)
    del vec
    print(f"Inference took {round((ittime.time_ns() - start)/10e8, 5)} seconds")
    start = ittime.time_ns()
    print(f"Everything took {round((ittime.time_ns() - first_start)/10e8, 5)} seconds")

--------------------------------------------------------------------
0.35Go 00c058_0.tif
Openslide took 0.02754 seconds
Preview took 99.52381 seconds
Clean took 11.50682 seconds
815
Inference took 60.72802 seconds
Everything took 171.7865 seconds
--------------------------------------------------------------------
0.68Go 01adc5_0.tif
Openslide took 0.02722 seconds
Preview took 176.15111 seconds
Clean took 0.19987 seconds
724
Inference took 31.56755 seconds
Everything took 207.946 seconds
--------------------------------------------------------------------
1.31Go 006388_0.tif
Openslide took 0.03681 seconds
Preview took 185.87645 seconds
Clean took 0.18237 seconds
3905
Inference took 167.15354 seconds
Everything took 353.24945 seconds
--------------------------------------------------------------------
0.11Go 008e5c_0.tif
Openslide took 0.01831 seconds
Preview took 22.96395 seconds
Clean took 0.02763 seconds
70
Inference took 7.89081 seconds
Everything took 30.90096 seconds


In [25]:
labelization

,patient_id,CE,LAA
2,006388,0.99999,0.00001
3,008e5c,0.99999,0.00001
0,00c058,0.99999,0.00001
1,01adc5,0.00001,0.99999
